<br>

# Introdução

In [1]:
import os
import re
import time
import random
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

from bs4 import BeautifulSoup
from osgeo import gdal, osr
from tqdm.notebook import trange, tqdm

<br>

# Get List Links

In [2]:
# Get Data
url = 'https://www.defesa.agricultura.sp.gov.br/enderecos/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
soup = soup.find('select',  attrs={'id':'local_mun'})
rows = soup.find_all('option')

In [3]:
# Listas Vazias
list_url = []
list_name = []

# Append as Listas
for row in rows:    
    list_name.append(row.get_text())
    list_url.append(row['value'])

# Lista de Tuplas
list_tuples = list(zip(list_name, list_url))[1:]

In [4]:
# Convert to Dataframe
df = pd.DataFrame(list_tuples, columns = ['municipio', 'url'])
df['url'] = url + df['url'] + '/'

# 
list_df_urls = df.iterrows()
df

,municipio,url
0,Adamantina,https://www.defesa.agricultura.sp.gov.br/ender...
1,Adolfo,https://www.defesa.agricultura.sp.gov.br/ender...
2,Aguaí,https://www.defesa.agricultura.sp.gov.br/ender...
3,Águas da Prata,https://www.defesa.agricultura.sp.gov.br/ender...
4,Águas de Lindóia,https://www.defesa.agricultura.sp.gov.br/ender...
...,...,...
640,Vista Alegre do Alto,https://www.defesa.agricultura.sp.gov.br/ender...
641,Vitória Brasil,https://www.defesa.agricultura.sp.gov.br/ender...
642,Votorantim,https://www.defesa.agricultura.sp.gov.br/ender...
643,Votuporanga,https://www.defesa.agricultura.sp.gov.br/ender...


<br>

# Get Data for Município

<br>

## Function

In [5]:
def get_data(municipio, url):
    # Get Data
    #url = 'https://www.defesa.agricultura.sp.gov.br/enderecos/motuca'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    soup = soup.find('form',  attrs={'name':'form2'}).find_next_sibling('div')

    # List
    list_data = soup.find('p').get_text().split('\n')
    dict_data = {
        'municipio_nome': municipio,
        'ida': soup.find('h2').text,
        'endereco': ([x for x in list_data if x.startswith('Endereço:')] or [''])[0],
        'municipio_sede': ([x for x in list_data if x.startswith('Município:')] or [''])[0],
        'telefone': ([x for x in list_data if x.startswith('Telefone:')] or [''])[0],
        'fax': ([x for x in list_data if x.startswith('Fax:')] or [''])[0],
        'email': ([x for x in list_data if x.startswith('E-mail:')] or [''])[0],
        'unidaderegional': ([x for x in list_data if x.startswith('Unidade Regional:')] or [''])[0],
        'diretor': ([x for x in list_data if x.startswith('Diretor:')] or [''])[0].replace('\r', ''),
        'url': url,
    }

    return dict_data

In [6]:
url = 'https://www.defesa.agricultura.sp.gov.br/enderecos/pedra-bela/'
get_data('Mombuca', url)

{'municipio_nome': 'Mombuca',
 'ida': 'Inspetoria de Defesa Agropecuária de Socorro',
 'endereco': 'Endereço: Av. Dr. Rebouças, 116 - CEP: 13960000 ',
 'municipio_sede': 'Município: Socorro/SP',
 'telefone': 'Telefone: (19) 3855-2011',
 'fax': '',
 'email': 'E-mail: ida.socorro@cda.sp.gov.br',
 'unidaderegional': 'Unidade Regional: EDA de Bragança Paulista',
 'diretor': 'Diretor: Eng. Agr. RogÉrio MarÇal Rocha Oliveira',
 'url': 'https://www.defesa.agricultura.sp.gov.br/enderecos/pedra-bela/'}

<br>

## Loop

In [7]:
list_dicts = []
for index, row in list_df_urls:
    # 
    municipio = row['municipio']
    url = row['url']

    # 
    dict_temp = get_data(municipio, url)
    list_dicts.append(dict_temp)

<br>

# Create Dataframe

In [8]:
# Create dataframe
df = pd.DataFrame(list_dicts)

In [9]:
# Divide Células em Duas Colunas
df[['endereco','cep']] = df['endereco'].str.split('- CEP:', expand=True)
df[['municipio_sede','estado']] = df['municipio_sede'].str.split('/', expand=True)
df[['tel_1','tel_2']] = df['telefone'].str.split('/', expand=True)
df[['fax_1','fax_2']] = df['fax'].str.split('/', expand=True)

In [10]:
# Remove tudo que vem antes do :
df = df.applymap(lambda x: x.split(':')[-1], na_action='ignore')

# Strip All Data
df = df.applymap(lambda x: x.strip(), na_action='ignore')

<br>

## Rename

In [11]:
def rename_nome(x):
    #x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '  '  : ' ',
        
        # Basics
        # ' Sobre ': ' sobre ',
        # ' Da '   : ' da ',
        # ' De '   : ' de ',
        # ' Do '   : ' do ',
        # ' Das '  : ' das ',
        # ' Dos '  : ' dos ',
        # ' A '    : ' a ',
        # ' As '   : ' as ',
        # ' Ao '   : ' ao ',
        # ' Aos '  : ' aos ',
        # ' E '    : ' e ',
        # ' O '    : ' o ',
        # ' Os '   : ' os ',
        
        # Erros
        '()'  : '',
        'Av. '  : 'Avenida ',
        'R. '  : 'Rua ',
        
        # Abreviações
        'Mogi-Mirim' : 'Mogi Mirim',
        's/nº': 's/n',
        's/n°': 's/n',
        's/n': 's/n',
        'S/N' : 's/n',
        'S/n' : 's/n',
        's/n' : 's/n',
        
        ' nº ': ', ',
        ', nº ': ', ',
        ', ,': ',',
        ',,': ',',

        'EDA de ': 'Escritório de Defesa Agropecuária de ',

        # 
        'RogÉrio': 'Rogério',
        'JosÉ': 'José',
        'CÉsar': 'César',
        'MarÇal': 'Marçal',
        'AdÃo': 'Adão',
        'TristÃo': 'Tristão',
        'ConceiÇÃo': 'Conceição',
        
        'Avednida ':'Avenida ',
        'Av.25 de Janeiro':'Avenida 25 de Janeiro',
        'Rua 13 de maio, 260': 'Rua 13 de Maio, 260',
        ' Do ': ' do ',
        'RUA FUAD EID,319': 'Rua Fuad Eid, 319',

    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [12]:
df.loc[:, 'ida'] = df['ida'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'endereco'] = df['endereco'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'municipio_sede'] = df['municipio_sede'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'tel_1'] = df['tel_1'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'unidaderegional'] = df['unidaderegional'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'diretor'] = df['diretor'].astype(str).apply(lambda x: rename_nome(x))

In [13]:
rename_nome('S/nº')

's/nº'

In [14]:
# Adjust Error
df.set_index('municipio_nome', inplace=True)
df.loc['Pontes Gestal', 'ida'] = 'Unidade de Defesa Agropecuária de Pontes Gestal'
df.reset_index(inplace=True)

In [15]:
# Fix Municípios
df['municipio_nome'] = df['municipio_nome'].replace('Embu', 'Embu das Artes').replace('Mogi-Mirim', 'Mogi Mirim')

# Fix Telefone
df['tel_1'] = df['tel_1'].replace('(11)', '').replace('(18)', '')

In [16]:
# Preenche espaços em branco por NaN
df.loc[:, 'unidaderegional'] = df['unidaderegional'].replace(r'^\s*$', np.nan, regex=True)

# Define máscara
mask = df['unidaderegional'].isnull()

# Para os que estão em branco, define que é igual ao ida
df.loc[mask, 'unidaderegional'] = df['ida']

In [17]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipio_nome.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='municipio_nome',
    right_on='municipio_nome'
)

# Resultados
df.head()

,id_municipio,municipio_nome,ida,endereco,municipio_sede,telefone,fax,email,unidaderegional,diretor,url,cep,estado,tel_1,tel_2,fax_1,fax_2
0,3500105,Adamantina,Inspetoria de Defesa Agropecuária de Adamantina,"Rua Alameda Santa Cruz, 541",Adamantina,(18),(18) 3521-5466,,Escritório de Defesa Agropecuária de Dracena,Eng. Agr. Leonardo da Cruz Oliveira Junior,//www.defesa.agricultura.sp.gov.br/enderecos/a...,17800000,SP,,None,(18) 3521-5466,None
1,3500204,Adolfo,Inspetoria de Defesa Agropecuária de José Boni...,"Rua Piratininga, 309",José Bonifácio,(17) 3245-4664/3245-1183,(17) 3245-4664/3245-1183,ida.josebonifacio@cda.sp.gov.br,Escritório de Defesa Agropecuária de São José ...,Eng. Agr. Alexandre Paloschi,//www.defesa.agricultura.sp.gov.br/enderecos/a...,15200000,SP,(17) 3245-4664,3245-1183,(17) 3245-4664,3245-1183
2,3500303,Aguaí,Escritório de Defesa Agropecuária de São João ...,"Rua Teófilo Ribeiro de Andrade, 1060",São João da Boa Vista,(19) 3623-3612/3623-3823,(19) 3623-3612/3623-3823,eda.sjbv@sp.gov.br,Escritório de Defesa Agropecuária de São João ...,Eng. Agr. Rogério Marçal Rocha Oliveira,//www.defesa.agricultura.sp.gov.br/enderecos/a...,13870210,SP,(19) 3623-3612,3623-3823,(19) 3623-3612,3623-3823
3,3500402,Águas da Prata,Escritório de Defesa Agropecuária de São João ...,"Rua Teófilo Ribeiro de Andrade, 1060",São João da Boa Vista,(19) 3623-3612/3623-3823,(19) 3623-3612/3623-3823,eda.sjbv@sp.gov.br,Escritório de Defesa Agropecuária de São João ...,Eng. Agr. Rogério Marçal Rocha Oliveira,//www.defesa.agricultura.sp.gov.br/enderecos/a...,13870210,SP,(19) 3623-3612,3623-3823,(19) 3623-3612,3623-3823
4,3500501,Águas de Lindóia,Inspetoria de Defesa Agropecuária de Socorro,"Avenida Dr. Rebouças, 116",Socorro,(19) 3855-2011,,ida.socorro@cda.sp.gov.br,Escritório de Defesa Agropecuária de Bragança ...,Eng. Agr. Rogério Marçal Rocha Oliveira,//www.defesa.agricultura.sp.gov.br/enderecos/a...,13960000,SP,(19) 3855-2011,None,,None


<br>

## Ordena Colunas

In [18]:
print(list(df.columns))

df = df[[
    'id_municipio',
    'municipio_nome',
    'ida',
    'endereco',
    'cep',
    'municipio_sede',
    'tel_1',
    'tel_2',
    'fax_1',
    'fax_2',
    'email',
    'unidaderegional',
    'diretor',
    'url',
]]

['id_municipio', 'municipio_nome', 'ida', 'endereco', 'municipio_sede', 'telefone', 'fax', 'email', 'unidaderegional', 'diretor', 'url', 'cep', 'estado', 'tel_1', 'tel_2', 'fax_1', 'fax_2']


<br>

# Salva

In [19]:
# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_eda.csv'),
    index=False
)